In [1]:
from neo4j import GraphDatabase

In [2]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [3]:
uri = 'bolt://54.91.36.152:7687'
pwd = 'scratches-specialty-jigs'

conn = Neo4jConnection(uri=uri, user="neo4j", pwd=pwd)

conn.query('MATCH (n) RETURN COUNT(n)')

[<Record COUNT(n)=2642>]

In [4]:
query = """CALL gds.graph.create(
           'interactions',
           'Person',
           {INTERACTS: 
               {
                   orientation: 'UNDIRECTED'
               }
           }
        )
"""

conn.query(query)

[<Record nodeProjection={'Person': {'properties': {}, 'label': 'Person'}} relationshipProjection={'INTERACTS': {'orientation': 'UNDIRECTED', 'aggregation': 'DEFAULT', 'type': 'INTERACTS', 'properties': {}}} graphName='interactions' nodeCount=2166 relationshipCount=7814 createMillis=32>]

In [5]:
query = """CALL gds.graph.create(
           'weighted-interactions',
           'Person',
           {INTERACTS: 
               {
                   orientation: 'UNDIRECTED',
                   properties: 'weight'
               }
           }
        )
"""

conn.query(query)

[<Record nodeProjection={'Person': {'properties': {}, 'label': 'Person'}} relationshipProjection={'INTERACTS': {'orientation': 'UNDIRECTED', 'aggregation': 'DEFAULT', 'type': 'INTERACTS', 'properties': {'weight': {'property': 'weight', 'aggregation': 'DEFAULT', 'defaultValue': None}}}} graphName='weighted-interactions' nodeCount=2166 relationshipCount=7814 createMillis=50>]

In [6]:
query = """CALL gds.louvain.stream(
            'weighted-interactions',
            {
                relationshipWeightProperty: 'weight'
            }
        )
        YIELD nodeId, communityId
        RETURN communityId, COUNT(DISTINCT nodeId) AS members
        ORDER BY members DESC
        LIMIT 5
"""

conn.query(query)

[<Record communityId=205 members=142>,
 <Record communityId=77 members=114>,
 <Record communityId=1202 members=104>,
 <Record communityId=233 members=86>,
 <Record communityId=15 members=85>]

In [7]:
query = """CALL gds.louvain.write(
           'interactions', 
               {
                   writeProperty: 'louv_community'
               }
        ) 
        YIELD communityCount, modularity, modularities
"""

conn.query(query)

[<Record communityCount=1382 modularity=0.5680039220030543 modularities=[0.4857842129222886, 0.5660974254959941, 0.5680039220030543]>]

In [8]:
query = """CALL gds.louvain.write(
           'weighted-interactions', 
               {
                   writeProperty: 'louv_community_weighted',
                   relationshipWeightProperty: 'weight'
               }
        ) 
        YIELD communityCount, modularity, modularities
"""

conn.query(query)

[<Record communityCount=1384 modularity=0.5693787565392012 modularities=[0.5371223276027431, 0.5693787565392012]>]